# Tutorial 02-FF — Train a SOEN Model with Forward-Forward Learning

This tutorial demonstrates **Forward-Forward (FF) learning** as an alternative to backpropagation for training SOEN networks.

**Architecture**: Matches `02_train_a_model.ipynb` with **5 hidden neurons** (same as SOEN SingleDendrite layer).

---

## What is Forward-Forward Learning?

Forward-Forward (Hinton, 2022) is a **local learning algorithm** that eliminates backpropagation:

| Aspect | Backpropagation | Forward-Forward |
|--------|-----------------|------------------|
| **Gradient flow** | Through all layers | None between layers |
| **Learning signal** | Global error | Local "goodness" |
| **Weight transport** | Required | Not needed |
| **Hardware fit** | Poor | Good |

---

## How Forward-Forward Works

```
┌─────────────────────────────────────────────────────────────────┐
│                    FORWARD-FORWARD LEARNING                     │
├─────────────────────────────────────────────────────────────────┤
│                                                                 │
│  POSITIVE PASS: Input + CORRECT label → Maximize "goodness"    │
│  NEGATIVE PASS: Input + WRONG label   → Minimize "goodness"    │
│                                                                 │
│  Each layer learns INDEPENDENTLY with its own local objective  │
│                                                                 │
└─────────────────────────────────────────────────────────────────┘
```

---

## ML Task: Pulse Classification

Same task as Tutorial 02:
- **Class 0**: Input contains a single pulse
- **Class 1**: Input contains two distinct pulses

---

## Architecture Matching

| Component | SOEN (02_train_a_model) | FF (This Tutorial) |
|-----------|-------------------------|---------------------|
| Hidden neurons | 5 | **5** |
| Input | 64 timesteps × 1 | 64 + 2 (label) = 66 |
| Output | 2 classes | 2 classes |

---

## 🔊 Noise Configuration

| Parameter | Default | Description |
|-----------|---------|-------------|
| `phi` | **0.01** | Noise on input flux |
| `s` | **0.005** | Noise on state |
| `relative` | **False** | Absolute scaling |

## Step 1: Setup and Imports

We import the necessary libraries and set up the environment.

In [ ]:
# ==============================================================================
# STEP 1: IMPORTS AND SETUP
# ==============================================================================
# This cell sets up the Python environment and imports all required libraries.
#
# Key imports:
#   - torch: PyTorch for tensor operations and neural networks
#   - numpy: Numerical operations
#   - h5py: Reading HDF5 dataset files
#   - matplotlib: Visualization
# ==============================================================================

import sys
from pathlib import Path

# Add src directory to path if running from notebook location
notebook_dir = Path.cwd()
for parent in [notebook_dir] + list(notebook_dir.parents):
    candidate = parent / "src"
    if (candidate / "soen_toolkit").exists():
        sys.path.insert(0, str(candidate))
        break

import numpy as np
import matplotlib.pyplot as plt
import h5py
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from typing import List, Dict, Tuple, Optional
from dataclasses import dataclass
from tqdm import tqdm

# Set device
DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"PyTorch version: {torch.__version__}")
print(f"Using device: {DEVICE}")

## Step 2: Configuration

We define all hyperparameters and settings in a single configuration class.

### Hardware Mode Options

| Mode | Description | Accuracy | Hardware Ready |
|------|-------------|----------|----------------|
| `HARDWARE_MODE = False` | Uses autograd + Adam | Higher | No |
| `HARDWARE_MODE = True` | Uses Hebbian learning | Lower | Yes |

In [ ]:
# ==============================================================================
# STEP 2: CONFIGURATION
# ==============================================================================
# All hyperparameters and settings are defined here.
# This makes it easy to experiment with different configurations.
#
# IMPORTANT: Architecture matches the non-FF version (02_train_a_model.ipynb)
#   - Hidden dim = 5 (same as 1D_5D_2D_PulseNetSpec_trainable.yaml)
#   - Single hidden layer (matching SOEN architecture)
#
# Key settings:
#   - HARDWARE_MODE: If True, uses hardware-compatible Hebbian learning
#   - NOISE_ENABLED: If True, adds noise during training (more realistic)
#   - hidden_dim: Number of neurons in hidden layer (5 to match SOEN)
#   - threshold: Goodness threshold for FF learning
# ==============================================================================

# Toggle between standard FF (autograd) and hardware-compatible FF (Hebbian)
HARDWARE_MODE = False  # Set to True for hardware-compatible mode

# Toggle noise injection
NOISE_ENABLED = True

@dataclass
class FFConfig:
    """Configuration for Forward-Forward training on pulse classification.
    
    Architecture matches the SOEN model from 02_train_a_model.ipynb:
        Input (1) → Hidden (5) → Output (2)
    
    For FF, we flatten the sequence and concatenate the label:
        Input: seq_len * input_dim + num_classes = 64 * 1 + 2 = 66
    """
    
    # Dataset
    data_path: str = "training/datasets/soen_seq_task_one_or_two_pulses_seq64.hdf5"
    seq_len: int = 64          # Sequence length (timesteps)
    input_dim: int = 1         # Input features per timestep
    num_classes: int = 2       # Binary classification (one vs two pulses)
    
    # Architecture - MATCHING THE SOEN MODEL (1D → 5D → 2D)
    # The SOEN model uses dim=5 for the hidden SingleDendrite layer
    hidden_dim: int = 5        # Hidden layer size (matches SOEN!)
    num_layers: int = 1        # Number of FF layers (matches single SOEN hidden layer)
    
    # Training
    batch_size: int = 32
    num_epochs: int = 100      # More epochs needed for smaller network
    learning_rate: float = 0.03     # For standard mode (Adam)
    hebbian_lr: float = 0.1         # For hardware mode (Hebbian)
    
    # Forward-Forward specific
    threshold: float = 2.0     # Goodness threshold
    
    # Hardware mode settings
    hardware_mode: bool = HARDWARE_MODE
    inhibition_strength: float = 0.1  # Lateral inhibition for hardware mode
    
    # Noise
    noise_enabled: bool = NOISE_ENABLED
    noise_phi: float = 0.01    # Noise on input flux
    noise_s: float = 0.005     # Noise on state
    
    @property
    def flattened_input_dim(self) -> int:
        """Total input dimension after flattening sequence + adding label."""
        return self.seq_len * self.input_dim + self.num_classes  # 64 + 2 = 66


# Create configuration
config = FFConfig()

# Calculate parameter counts for comparison
ff_params = (config.flattened_input_dim * config.hidden_dim +  # Input weights
             config.hidden_dim +                                # Input biases
             config.hidden_dim * config.hidden_dim +            # Recurrent (if num_layers > 1)
             config.hidden_dim)                                 # Additional biases
soen_params = (1 * 5 +        # J_0_to_1: 1→5
               5 * 5 - 5 +    # J_1_to_1: 5→5 no self = 20
               2)             # J_1_to_2: one_to_one ≈ 2

# Print configuration
print("="*60)
print("FORWARD-FORWARD CONFIGURATION")
print("="*60)
print(f"Hardware mode: {config.hardware_mode}")
print(f"Noise enabled: {config.noise_enabled}")
print(f"")
print(f"Dataset: {config.data_path}")
print(f"Sequence length: {config.seq_len}")
print(f"Input dim: {config.input_dim}")
print(f"Flattened + label dim: {config.flattened_input_dim}")
print(f"")
print(f"╔══════════════════════════════════════════════════════════╗")
print(f"║  ARCHITECTURE (matching SOEN model)                      ║")
print(f"╠══════════════════════════════════════════════════════════╣")
print(f"║  Hidden dim: {config.hidden_dim:3d} neurons (same as SOEN SingleDendrite) ║")
print(f"║  Num layers: {config.num_layers:3d} (same as SOEN hidden layers)         ║")
print(f"║  Num classes: {config.num_classes:2d}                                      ║")
print(f"╚══════════════════════════════════════════════════════════╝")
print(f"")
print(f"Batch size: {config.batch_size}")
print(f"Epochs: {config.num_epochs}")
print(f"Learning rate: {config.hebbian_lr if config.hardware_mode else config.learning_rate}")
print(f"Threshold: {config.threshold}")
print(f"")
print(f"Approx SOEN params: ~{soen_params} connection weights")
print(f"Approx FF params:   ~{(config.flattened_input_dim + 1) * config.hidden_dim} weights")
print("="*60)

## Step 3: Load and Visualize Dataset

We load the pulse classification dataset and visualize examples from each class.

### Dataset Structure
- **Shape**: `(N, T, D)` = `(samples, timesteps, features)`
- **Class 0**: Single pulse in the signal
- **Class 1**: Two pulses in the signal

In [ ]:
# ==============================================================================
# STEP 3: LOAD AND VISUALIZE DATASET
# ==============================================================================
# This cell loads the HDF5 dataset and visualizes examples from each class.
#
# The dataset contains:
#   - Class 0: Signals with ONE pulse
#   - Class 1: Signals with TWO pulses
#
# Dataset format:
#   - data: shape (N, T, D) where N=samples, T=timesteps, D=features
#   - labels: shape (N,) with integer class labels
# ==============================================================================

def load_dataset(data_path: str) -> Tuple[np.ndarray, np.ndarray, np.ndarray, np.ndarray]:
    """
    Load train and validation data from HDF5 file.
    
    Args:
        data_path: Path to HDF5 dataset file
        
    Returns:
        train_data, train_labels, val_data, val_labels
    """
    with h5py.File(data_path, 'r') as f:
        # Load training data
        train_data = np.array(f['train']['data'])
        train_labels = np.array(f['train']['labels'])
        
        # Load validation data (or use part of train if not available)
        if 'val' in f:
            val_data = np.array(f['val']['data'])
            val_labels = np.array(f['val']['labels'])
        else:
            # Split train data
            split_idx = int(len(train_data) * 0.8)
            val_data = train_data[split_idx:]
            val_labels = train_labels[split_idx:]
            train_data = train_data[:split_idx]
            train_labels = train_labels[:split_idx]
    
    return train_data, train_labels, val_data, val_labels


def visualize_dataset(data: np.ndarray, labels: np.ndarray, n_examples: int = 4):
    """
    Visualize examples from each class.
    
    Args:
        data: Input data of shape (N, T, D)
        labels: Class labels of shape (N,)
        n_examples: Number of examples per class to show
    """
    print(f"Dataset shape: {data.shape} (N samples, T timesteps, D features)")
    print(f"Labels shape: {labels.shape}")
    print(f"Class distribution: {np.bincount(labels)}")
    
    # Find examples of each class
    class_0_idx = np.where(labels == 0)[0][:n_examples]
    class_1_idx = np.where(labels == 1)[0][:n_examples]
    
    fig, axes = plt.subplots(2, n_examples, figsize=(3*n_examples, 5))
    fig.suptitle("Input Signals: One-Pulse (Class 0) vs Two-Pulse (Class 1)", 
                 fontsize=12, fontweight='bold')
    
    # Plot Class 0 (single pulse)
    for i, idx in enumerate(class_0_idx):
        axes[0, i].plot(data[idx, :, 0], 'b-', linewidth=1.5)
        axes[0, i].set_title(f"Sample {idx}", fontsize=10)
        axes[0, i].set_ylim(-0.1, 1.1)
        axes[0, i].grid(True, alpha=0.3)
        if i == 0:
            axes[0, i].set_ylabel("Class 0\n(One Pulse)", fontsize=10)
    
    # Plot Class 1 (two pulses)
    for i, idx in enumerate(class_1_idx):
        axes[1, i].plot(data[idx, :, 0], 'r-', linewidth=1.5)
        axes[1, i].set_title(f"Sample {idx}", fontsize=10)
        axes[1, i].set_ylim(-0.1, 1.1)
        axes[1, i].grid(True, alpha=0.3)
        if i == 0:
            axes[1, i].set_ylabel("Class 1\n(Two Pulses)", fontsize=10)
        axes[1, i].set_xlabel("Time step")
    
    plt.tight_layout()
    plt.show()


# Load dataset
train_data, train_labels, val_data, val_labels = load_dataset(config.data_path)

print(f"\nTrain set: {train_data.shape[0]} samples")
print(f"Val set: {val_data.shape[0]} samples")

# Visualize
visualize_dataset(train_data, train_labels)

## Step 4: Create Forward-Forward Dataset

For Forward-Forward learning, we need to create **positive** and **negative** examples:

```
┌─────────────────────────────────────────────────────────────────┐
│  POSITIVE: [flattened signal (64)] + [correct label one-hot]   │
│  NEGATIVE: [flattened signal (64)] + [wrong label one-hot]     │
│                                                                 │
│  Total dimension: 64 + 2 = 66                                   │
└─────────────────────────────────────────────────────────────────┘
```

The label is **concatenated** (not replacing signal values) to preserve all information.

In [ ]:
# ==============================================================================
# STEP 4: CREATE FORWARD-FORWARD DATASET
# ==============================================================================
# This cell creates a PyTorch Dataset that returns positive and negative pairs.
#
# For Forward-Forward learning:
#   - POSITIVE: signal + correct label → should have HIGH goodness
#   - NEGATIVE: signal + wrong label → should have LOW goodness
#
# Label embedding:
#   - We CONCATENATE the one-hot label to the flattened signal
#   - This preserves all signal information (unlike pixel replacement)
#   - Input dim = seq_len * input_dim + num_classes = 64 + 2 = 66
# ==============================================================================

class PulseFFDataset(Dataset):
    """
    Pulse classification dataset for Forward-Forward learning.
    
    Each sample returns:
        - positive: flattened signal + correct label one-hot
        - negative: flattened signal + wrong label one-hot
        - label: true class label
        - signal: original signal for visualization
    """
    
    def __init__(self, data: np.ndarray, labels: np.ndarray, num_classes: int = 2):
        """
        Args:
            data: Signal data of shape (N, T, D)
            labels: Class labels of shape (N,)
            num_classes: Number of classes (2 for pulse classification)
        """
        # Convert to tensors
        self.data = torch.tensor(data, dtype=torch.float32)
        self.labels = torch.tensor(labels, dtype=torch.long)
        self.num_classes = num_classes
        
        # Pre-flatten signals for efficiency
        # Shape: (N, T, D) -> (N, T*D)
        self.flat_data = self.data.view(len(data), -1)
        
        print(f"Created FF Dataset:")
        print(f"  Original shape: {self.data.shape}")
        print(f"  Flattened shape: {self.flat_data.shape}")
        print(f"  + label one-hot: {num_classes}")
        print(f"  = Total input dim: {self.flat_data.shape[1] + num_classes}")
    
    def __len__(self) -> int:
        return len(self.labels)
    
    def _embed_label(self, flat_signal: torch.Tensor, label: int) -> torch.Tensor:
        """
        Embed label by CONCATENATING one-hot to flattened signal.
        
        Args:
            flat_signal: Flattened signal of shape (T*D,)
            label: Integer class label
            
        Returns:
            Embedded tensor of shape (T*D + num_classes,)
        """
        # Create one-hot label
        one_hot = F.one_hot(torch.tensor(label), num_classes=self.num_classes).float()
        
        # Concatenate signal + label
        return torch.cat([flat_signal, one_hot], dim=0)
    
    def _get_wrong_label(self, true_label: int) -> int:
        """
        Get a wrong label for negative example.
        
        For binary classification: just flip the label.
        For multi-class: randomly select a different label.
        """
        if self.num_classes == 2:
            return 1 - true_label  # Flip: 0->1, 1->0
        else:
            wrong = torch.randint(0, self.num_classes - 1, (1,)).item()
            if wrong >= true_label:
                wrong += 1
            return wrong
    
    def __getitem__(self, idx: int) -> Dict[str, torch.Tensor]:
        """
        Get a sample with positive and negative versions.
        
        Returns:
            Dictionary with:
                - positive: signal + correct label (should have HIGH goodness)
                - negative: signal + wrong label (should have LOW goodness)
                - label: true class label
                - signal: original signal for visualization
        """
        flat_signal = self.flat_data[idx]
        true_label = self.labels[idx].item()
        wrong_label = self._get_wrong_label(true_label)
        
        return {
            "positive": self._embed_label(flat_signal, true_label),
            "negative": self._embed_label(flat_signal, wrong_label),
            "label": self.labels[idx],
            "signal": self.data[idx],
        }


# Create datasets
print("Creating training dataset...")
train_dataset = PulseFFDataset(train_data, train_labels, num_classes=config.num_classes)

print("\nCreating validation dataset...")
val_dataset = PulseFFDataset(val_data, val_labels, num_classes=config.num_classes)

# Create data loaders
train_loader = DataLoader(train_dataset, batch_size=config.batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=config.batch_size, shuffle=False)

# Verify a sample
print("\n" + "="*60)
print("SAMPLE VERIFICATION")
print("="*60)
sample = train_dataset[0]
print(f"Positive shape: {sample['positive'].shape}")
print(f"Negative shape: {sample['negative'].shape}")
print(f"True label: {sample['label'].item()}")
print(f"")
print(f"Signal part (first 5 values): {sample['positive'][:5].tolist()}")
print(f"Label part (last 2 values):")
print(f"  Positive (correct): {sample['positive'][-2:].tolist()}")
print(f"  Negative (wrong):   {sample['negative'][-2:].tolist()}")

## Step 5: Define Forward-Forward Layer

Each FF layer has its own **local learning objective**:

```
┌─────────────────────────────────────────────────────────────────┐
│  GOODNESS = measure of layer activity                          │
│                                                                 │
│  Goal: positive_goodness > threshold > negative_goodness       │
│                                                                 │
│  Standard mode:  goodness = mean(x²),     loss = softplus      │
│  Hardware mode:  goodness = mean(|x|),    loss = hinge         │
└─────────────────────────────────────────────────────────────────┘
```

In [ ]:
# ==============================================================================
# STEP 5: DEFINE FORWARD-FORWARD LAYER
# ==============================================================================
# This cell defines the FFLayer class - the building block of our network.
#
# Each layer has:
#   - A linear transformation (weights and biases)
#   - A ReLU activation
#   - A normalization step
#   - A "goodness" computation for the FF objective
#
# The layer learns to:
#   - Have HIGH goodness for positive data (correct label)
#   - Have LOW goodness for negative data (wrong label)
#
# Two modes are supported:
#   - Standard: Uses autograd for gradient computation
#   - Hardware: Uses Hebbian learning (gradient-free)
# ==============================================================================

class FFLayer(nn.Module):
    """
    Forward-Forward Layer with local learning objective.
    
    This layer learns to distinguish between positive (correct label)
    and negative (wrong label) inputs using a local "goodness" measure.
    
    Architecture:
        Input -> Linear -> ReLU -> Normalize -> Output
                                      |
                                 Compute Goodness
                                      |
                                 Local Learning
    """
    
    def __init__(
        self,
        in_features: int,
        out_features: int,
        threshold: float = 2.0,
        hardware_mode: bool = False,
        inhibition_strength: float = 0.1,
    ):
        """
        Args:
            in_features: Input dimension
            out_features: Output dimension (number of neurons)
            threshold: Goodness threshold for FF learning
            hardware_mode: If True, use hardware-compatible operations
            inhibition_strength: Strength of lateral inhibition (hardware mode)
        """
        super().__init__()
        
        self.in_features = in_features
        self.out_features = out_features
        self.threshold = threshold
        self.hardware_mode = hardware_mode
        self.inhibition_strength = inhibition_strength
        
        # Linear transformation: y = Wx + b
        self.linear = nn.Linear(in_features, out_features)
        
        # ReLU activation
        self.relu = nn.ReLU()
        
        # Initialize weights
        if hardware_mode:
            # Larger initialization for hardware mode
            nn.init.uniform_(self.linear.weight, -0.3, 0.3)
            nn.init.zeros_(self.linear.bias)
        
        # Storage for activations (needed for Hebbian learning)
        self._raw_activations = None
        self._pos_input = None
        self._pos_activations = None
        self._neg_input = None
        self._neg_activations = None
        
        # Running estimates for adaptive threshold
        self._running_pos_goodness = 0.1
        self._running_neg_goodness = 0.1
        self._momentum = 0.95
    
    def forward(self, x: torch.Tensor, store_as: str = None) -> torch.Tensor:
        """
        Forward pass through the layer.
        
        Args:
            x: Input tensor of shape (batch, in_features)
            store_as: If 'positive' or 'negative', store activations for Hebbian learning
            
        Returns:
            Normalized output tensor of shape (batch, out_features)
        """
        # Store input for Hebbian learning
        if store_as == 'positive':
            self._pos_input = x.detach()
        elif store_as == 'negative':
            self._neg_input = x.detach()
        
        # Linear transformation
        x = self.linear(x)
        
        # ReLU activation
        x = self.relu(x)
        
        # Store raw activations for goodness computation
        self._raw_activations = x
        
        # Store activations for Hebbian learning
        if store_as == 'positive':
            self._pos_activations = x.detach()
        elif store_as == 'negative':
            self._neg_activations = x.detach()
        
        # Normalization
        if self.hardware_mode:
            # Hardware-compatible: Lateral inhibition
            # Subtract mean activity (can be implemented with differential amplifier)
            mean_activity = x.mean(dim=1, keepdim=True)
            x_norm = x - self.inhibition_strength * mean_activity
            x_norm = torch.clamp(x_norm, min=0)
        else:
            # Standard: L2 normalization
            x_norm = x / (x.norm(dim=1, keepdim=True) + 1e-8)
        
        return x_norm
    
    def compute_goodness(self) -> torch.Tensor:
        """
        Compute "goodness" of the current activations.
        
        Goodness measures how "active" the layer is:
            - Standard mode: mean(x²) - requires squaring circuit
            - Hardware mode: mean(|x|) - uses rectifiers only
            
        Returns:
            Goodness value per sample, shape (batch,)
        """
        activations = self._raw_activations
        
        if self.hardware_mode:
            # Hardware-compatible: mean absolute value
            # Can be computed with full-wave rectifier
            return activations.abs().mean(dim=1)
        else:
            # Standard: mean squared value
            return (activations ** 2).mean(dim=1)
    
    def ff_loss(self, pos_goodness: torch.Tensor, neg_goodness: torch.Tensor) -> torch.Tensor:
        """
        Compute Forward-Forward loss.
        
        Goal: Push positive goodness ABOVE threshold, negative BELOW.
        
        Args:
            pos_goodness: Goodness for positive data, shape (batch,)
            neg_goodness: Goodness for negative data, shape (batch,)
            
        Returns:
            Scalar loss value
        """
        # Use adaptive threshold in hardware mode
        if self.hardware_mode:
            threshold = (self._running_pos_goodness + self._running_neg_goodness) / 2
        else:
            threshold = self.threshold
        
        if self.hardware_mode:
            # Hardware-compatible: Hinge loss
            # Uses comparators and ReLU (threshold circuits)
            pos_loss = F.relu(threshold - pos_goodness)  # Want pos > threshold
            neg_loss = F.relu(neg_goodness - threshold)  # Want neg < threshold
        else:
            # Standard: Softplus loss (smooth approximation)
            pos_loss = F.softplus(-(pos_goodness - threshold))
            neg_loss = F.softplus(neg_goodness - threshold)
        
        return (pos_loss + neg_loss).mean()
    
    def hebbian_update(self, pos_goodness: torch.Tensor, neg_goodness: torch.Tensor,
                       lr: float = 0.01) -> None:
        """
        Perform Hebbian weight update (gradient-free learning).
        
        This implements "differential Hebbian learning":
            - Compare activations for positive vs negative inputs
            - Strengthen connections that help distinguish them
        
        Args:
            pos_goodness: Goodness for positive data
            neg_goodness: Goodness for negative data
            lr: Hebbian learning rate
        """
        with torch.no_grad():
            if self._pos_input is None or self._neg_input is None:
                return
            
            batch_size = self._pos_input.shape[0]
            
            # Update running goodness estimates
            pos_g_mean = pos_goodness.mean().item()
            neg_g_mean = neg_goodness.mean().item()
            self._running_pos_goodness = self._momentum * self._running_pos_goodness + (1 - self._momentum) * pos_g_mean
            self._running_neg_goodness = self._momentum * self._running_neg_goodness + (1 - self._momentum) * neg_g_mean
            
            # Compute differences between positive and negative
            diff_acts = self._pos_activations - self._neg_activations  # Output difference
            diff_input = self._pos_input - self._neg_input              # Input difference (label part)
            avg_acts = (self._pos_activations + self._neg_activations) / 2
            avg_input = (self._pos_input + self._neg_input) / 2
            
            # Differential Hebbian update
            update1 = diff_acts.T @ avg_input / batch_size
            update2 = avg_acts.T @ diff_input / batch_size
            delta_w = lr * (update1 + 2.0 * update2)
            
            # Apply update
            self.linear.weight.data += delta_w
            
            # Update bias
            goodness_diff = pos_g_mean - neg_g_mean
            self.linear.bias.data += lr * 0.5 * goodness_diff
            
            # Clamp weights to hardware constraints
            self.linear.weight.data.clamp_(-0.5, 0.5)
            self.linear.bias.data.clamp_(-1.0, 1.0)


# Test the layer
print("Testing FFLayer...")
test_layer = FFLayer(66, 128, threshold=config.threshold, hardware_mode=config.hardware_mode)
test_input = torch.randn(32, 66)  # Batch of 32, input dim 66
test_output = test_layer(test_input)
test_goodness = test_layer.compute_goodness()

print(f"Input shape: {test_input.shape}")
print(f"Output shape: {test_output.shape}")
print(f"Goodness shape: {test_goodness.shape}")
print(f"Mean goodness: {test_goodness.mean().item():.4f}")

## Step 6: Define Forward-Forward Network

We stack FF layers to match the SOEN architecture from `02_train_a_model.ipynb`.

### Architecture Comparison

| Component | SOEN (Backprop) | FF (This Tutorial) |
|-----------|-----------------|---------------------|
| **Input processing** | Temporal (64 steps × 1 feature) | Flattened (64 + 2 label = 66) |
| **Hidden layer** | 5 SingleDendrite neurons | 5 FF neurons |
| **Output** | 2 classes | 2 classes |
| **Recurrence** | Yes (J_1_to_1) | No (feedforward only) |

```
┌─────────────────────────────────────────────────────────────────┐
│  INPUT: [flattened signal (64)] + [label one-hot (2)] = 66     │
│                          │                                      │
│                          ▼                                      │
│  ┌───────────────────────────────────────────────────────────┐ │
│  │ FF Layer 1: 66 → 5 neurons                                │ │
│  │   Linear → ReLU → Normalize → Goodness                   │ │
│  │   LOCAL OBJECTIVE: pos_goodness > θ > neg_goodness       │ │
│  └───────────────────────────────────────────────────────────┘ │
│                          │                                      │
│                          ▼                                      │
│  INFERENCE: Test each label, select highest total goodness     │
└─────────────────────────────────────────────────────────────────┘
```

**Key Difference**: SOEN processes the signal temporally with recurrence, while FF flattens
the entire sequence. This loses temporal structure but enables local learning.

In [ ]:
# ==============================================================================
# STEP 6: DEFINE FORWARD-FORWARD NETWORK
# ==============================================================================
# This cell defines the complete FF network by stacking multiple FF layers.
#
# Key concepts:
#   - Each layer learns INDEPENDENTLY (no gradient flow between layers)
#   - Inference requires multiple forward passes (one per class)
#   - Prediction = class with highest total goodness across all layers
#
# Architecture for pulse classification (matching SOEN):
#   Input (66) → FFLayer (5) → Prediction
#
# Note: The SOEN model processes temporally (64 timesteps × 1 feature),
# while FF flattens the sequence (64 features + 2 label = 66 input dim).
# ==============================================================================

class FFNetwork(nn.Module):
    """
    Forward-Forward Network for classification.
    
    This network stacks multiple FF layers, each trained with its own
    local objective. During inference, it performs multiple forward passes
    (one per class) and selects the class with highest total goodness.
    """
    
    def __init__(
        self,
        input_dim: int,
        hidden_dim: int,
        num_classes: int,
        num_layers: int = 1,
        threshold: float = 2.0,
        hardware_mode: bool = False,
    ):
        """
        Args:
            input_dim: Dimension of flattened input (without label)
            hidden_dim: Hidden layer dimension
            num_classes: Number of classes
            num_layers: Number of FF layers
            threshold: Goodness threshold
            hardware_mode: Use hardware-compatible operations
        """
        super().__init__()
        
        self.input_dim = input_dim
        self.num_classes = num_classes
        self.hardware_mode = hardware_mode
        
        # Total input dim = signal + label
        total_input_dim = input_dim + num_classes
        
        # Build FF layers
        self.layers = nn.ModuleList()
        
        # First layer: input -> hidden
        self.layers.append(FFLayer(
            total_input_dim, hidden_dim,
            threshold=threshold,
            hardware_mode=hardware_mode
        ))
        
        # Additional hidden layers (if any)
        for _ in range(num_layers - 1):
            self.layers.append(FFLayer(
                hidden_dim, hidden_dim,
                threshold=threshold,
                hardware_mode=hardware_mode
            ))
        
        print(f"\nCreated FF Network:")
        print(f"  Input dim: {input_dim} (signal) + {num_classes} (label) = {total_input_dim}")
        print(f"  Layers: {num_layers} x {hidden_dim} neurons")
        print(f"  Hardware mode: {hardware_mode}")
        print(f"  Total params: {sum(p.numel() for p in self.parameters()):,}")
    
    def forward_with_label(self, flat_signal: torch.Tensor, label: int,
                           store_as: str = None) -> torch.Tensor:
        """
        Forward pass with a specific label embedded.
        
        Args:
            flat_signal: Flattened input signal, shape (batch, input_dim)
            label: Class label to embed
            store_as: 'positive' or 'negative' for Hebbian learning
            
        Returns:
            Final layer output
        """
        batch_size = flat_signal.shape[0]
        device = flat_signal.device
        
        # Create one-hot label
        one_hot = F.one_hot(torch.tensor([label], device=device), self.num_classes)
        one_hot = one_hot.float().expand(batch_size, -1)
        
        # Concatenate signal + label
        x = torch.cat([flat_signal, one_hot], dim=1)
        
        # Forward through all layers
        for layer in self.layers:
            x = layer(x, store_as=store_as)
            if store_as is None:
                x = x.detach()  # No gradient flow between layers during inference
        
        return x
    
    def compute_total_goodness(self, flat_signal: torch.Tensor, label: int) -> torch.Tensor:
        """
        Compute total goodness across all layers for a given label.
        
        Args:
            flat_signal: Flattened input signal
            label: Class label to test
            
        Returns:
            Total goodness per sample, shape (batch,)
        """
        batch_size = flat_signal.shape[0]
        device = flat_signal.device
        
        # Create one-hot label
        one_hot = F.one_hot(torch.tensor([label], device=device), self.num_classes)
        one_hot = one_hot.float().expand(batch_size, -1)
        
        # Concatenate signal + label
        x = torch.cat([flat_signal, one_hot], dim=1)
        
        # Sum goodness across all layers
        total_goodness = torch.zeros(batch_size, device=device)
        
        for layer in self.layers:
            x = layer(x)
            total_goodness += layer.compute_goodness()
            x = x.detach()  # No gradient flow between layers
        
        return total_goodness
    
    def predict(self, flat_signal: torch.Tensor) -> torch.Tensor:
        """
        Predict class labels using multi-pass inference.
        
        For each class, we compute total goodness and select the class
        with the highest goodness.
        
        Args:
            flat_signal: Flattened input signal, shape (batch, input_dim)
            
        Returns:
            Predicted class labels, shape (batch,)
        """
        batch_size = flat_signal.shape[0]
        device = flat_signal.device
        
        # Compute goodness for each class
        all_goodness = torch.zeros(batch_size, self.num_classes, device=device)
        
        for label in range(self.num_classes):
            all_goodness[:, label] = self.compute_total_goodness(flat_signal, label)
        
        # Return class with highest goodness
        return all_goodness.argmax(dim=1)
    
    def get_layer_optimizer(self, layer_idx: int, lr: float) -> torch.optim.Optimizer:
        """
        Get optimizer for a specific layer (standard mode only).
        
        Args:
            layer_idx: Index of layer
            lr: Learning rate
            
        Returns:
            Adam optimizer for the layer's parameters
        """
        return torch.optim.Adam(self.layers[layer_idx].parameters(), lr=lr)


# Create network - using config.num_layers for architecture matching
model = FFNetwork(
    input_dim=config.seq_len * config.input_dim,  # 64
    hidden_dim=config.hidden_dim,                  # 5 (matching SOEN)
    num_classes=config.num_classes,                # 2
    num_layers=config.num_layers,                  # 1 (matching SOEN)
    threshold=config.threshold,
    hardware_mode=config.hardware_mode,
).to(DEVICE)

# Count parameters
total_params = sum(p.numel() for p in model.parameters())
print(f"\nTotal parameters: {total_params:,}")
print(f"\nNote: SOEN processes 64 timesteps sequentially with recurrence.")
print(f"      FF flattens the sequence, losing temporal structure.")
print(f"      This is a key architectural difference.")

## Step 7: Training Loop

The key difference from backpropagation:

| Backpropagation | Forward-Forward |
|-----------------|------------------|
| Global loss → backward() | Local loss per layer |
| One forward pass | Two passes (positive + negative) |
| Gradients flow through all layers | No gradients between layers |
| Single optimizer | One optimizer per layer (or Hebbian) |

In [ ]:
# ==============================================================================
# STEP 7: TRAINING LOOP
# ==============================================================================
# This cell implements the Forward-Forward training algorithm.
#
# Key differences from backpropagation:
#   1. Each layer is trained INDEPENDENTLY
#   2. Two forward passes per batch: positive and negative
#   3. No gradient flow between layers (outputs are detached)
#   4. Local loss function based on "goodness"
#
# Training flow:
#   For each batch:
#     For each layer:
#       1. Forward positive data → compute positive goodness
#       2. Forward negative data → compute negative goodness
#       3. Compute local loss: push pos > threshold > neg
#       4. Update layer weights (Adam or Hebbian)
#       5. Pass output to next layer (detached)
# ==============================================================================

def train_ff(
    model: FFNetwork,
    train_loader: DataLoader,
    val_loader: DataLoader,
    num_epochs: int,
    lr: float,
    hebbian_lr: float,
    device: torch.device,
) -> Dict:
    """
    Train using Forward-Forward algorithm.
    
    Args:
        model: FFNetwork to train
        train_loader: Training data loader
        val_loader: Validation data loader
        num_epochs: Number of training epochs
        lr: Learning rate for Adam (standard mode)
        hebbian_lr: Learning rate for Hebbian (hardware mode)
        device: Device to train on
        
    Returns:
        Training history dictionary
    """
    model.to(device)
    hardware_mode = model.hardware_mode
    
    # Create optimizers for each layer (standard mode only)
    if not hardware_mode:
        optimizers = [model.get_layer_optimizer(i, lr) for i in range(len(model.layers))]
    
    # Training history
    history = {
        "train_loss": [],
        "train_acc": [],
        "val_acc": [],
        "layer_losses": [[] for _ in range(len(model.layers))],
        "layer_pos_goodness": [[] for _ in range(len(model.layers))],
        "layer_neg_goodness": [[] for _ in range(len(model.layers))],
    }
    
    # Print training info
    mode_str = "HARDWARE (Hebbian)" if hardware_mode else "STANDARD (Autograd)"
    print(f"\n{'='*60}")
    print(f"FORWARD-FORWARD TRAINING - {mode_str}")
    print(f"{'='*60}")
    print(f"Epochs: {num_epochs}")
    print(f"Learning rate: {hebbian_lr if hardware_mode else lr}")
    print(f"Layers: {len(model.layers)}")
    print(f"Device: {device}")
    print(f"{'='*60}\n")
    
    # Training loop
    for epoch in range(num_epochs):
        model.train()
        
        # Track metrics
        epoch_losses = [0.0 for _ in range(len(model.layers))]
        epoch_pos_goodness = [0.0 for _ in range(len(model.layers))]
        epoch_neg_goodness = [0.0 for _ in range(len(model.layers))]
        num_batches = 0
        
        # Progress bar
        pbar = tqdm(train_loader, desc=f"Epoch {epoch+1}/{num_epochs}")
        
        for batch in pbar:
            # Get positive and negative data
            pos_data = batch["positive"].to(device)
            neg_data = batch["negative"].to(device)
            
            # Current inputs for each layer
            pos_input = pos_data
            neg_input = neg_data
            
            batch_loss = 0.0
            
            # Train each layer independently
            for layer_idx, layer in enumerate(model.layers):
                
                if hardware_mode:
                    # ========================================
                    # HARDWARE MODE: Hebbian learning
                    # ========================================
                    with torch.no_grad():
                        # Forward positive data
                        pos_output = layer(pos_input, store_as='positive')
                        pos_goodness = layer.compute_goodness()
                        
                        # Forward negative data
                        neg_output = layer(neg_input, store_as='negative')
                        neg_goodness = layer.compute_goodness()
                        
                        # Compute loss (for monitoring only)
                        loss = layer.ff_loss(pos_goodness, neg_goodness)
                    
                    # Apply Hebbian update
                    layer.hebbian_update(pos_goodness, neg_goodness, lr=hebbian_lr)
                    
                    # Prepare inputs for next layer
                    pos_input = pos_output.detach()
                    neg_input = neg_output.detach()
                    
                else:
                    # ========================================
                    # STANDARD MODE: Autograd + Adam
                    # ========================================
                    optimizer = optimizers[layer_idx]
                    optimizer.zero_grad()
                    
                    # Forward positive data
                    pos_output = layer(pos_input)
                    pos_goodness = layer.compute_goodness()
                    
                    # Forward negative data
                    neg_output = layer(neg_input)
                    neg_goodness = layer.compute_goodness()
                    
                    # Compute loss
                    loss = layer.ff_loss(pos_goodness, neg_goodness)
                    
                    # Backward pass (only for this layer!)
                    loss.backward()
                    optimizer.step()
                    
                    # Prepare inputs for next layer (detached = no gradient flow)
                    pos_input = pos_output.detach()
                    neg_input = neg_output.detach()
                
                # Track metrics
                epoch_losses[layer_idx] += loss.item()
                epoch_pos_goodness[layer_idx] += pos_goodness.mean().item()
                epoch_neg_goodness[layer_idx] += neg_goodness.mean().item()
                batch_loss += loss.item()
            
            num_batches += 1
            pbar.set_postfix({"loss": f"{batch_loss/len(model.layers):.4f}"})
        
        # Compute epoch averages
        for i in range(len(model.layers)):
            history["layer_losses"][i].append(epoch_losses[i] / num_batches)
            history["layer_pos_goodness"][i].append(epoch_pos_goodness[i] / num_batches)
            history["layer_neg_goodness"][i].append(epoch_neg_goodness[i] / num_batches)
        
        avg_loss = sum(epoch_losses) / (num_batches * len(model.layers))
        history["train_loss"].append(avg_loss)
        
        # Evaluate
        if (epoch + 1) % 5 == 0 or epoch == 0 or epoch == num_epochs - 1:
            train_acc = evaluate_ff(model, train_loader, device)
            val_acc = evaluate_ff(model, val_loader, device)
            history["train_acc"].append(train_acc)
            history["val_acc"].append(val_acc)
            
            print(f"\nEpoch {epoch+1}: Loss={avg_loss:.4f}, Train Acc={train_acc:.1%}, Val Acc={val_acc:.1%}")
            
            # Print layer-wise goodness
            for i in range(len(model.layers)):
                pos_g = history["layer_pos_goodness"][i][-1]
                neg_g = history["layer_neg_goodness"][i][-1]
                sep = pos_g - neg_g
                print(f"  Layer {i}: pos={pos_g:.3f}, neg={neg_g:.3f}, separation={sep:+.3f}")
    
    return history


def evaluate_ff(model: FFNetwork, data_loader: DataLoader, device: torch.device) -> float:
    """
    Evaluate Forward-Forward model accuracy.
    
    Args:
        model: Trained FFNetwork
        data_loader: Data loader for evaluation
        device: Device
        
    Returns:
        Accuracy (0 to 1)
    """
    model.eval()
    correct = 0
    total = 0
    
    with torch.no_grad():
        for batch in data_loader:
            # Get signal and labels
            signals = batch["signal"].to(device)
            labels = batch["label"].to(device)
            
            # Flatten signals
            flat_signals = signals.view(signals.shape[0], -1)
            
            # Predict
            predictions = model.predict(flat_signals)
            
            correct += (predictions == labels).sum().item()
            total += labels.size(0)
    
    return correct / total

In [ ]:
# ==============================================================================
# TRAIN THE MODEL
# ==============================================================================
# This cell runs the Forward-Forward training.
#
# Watch for:
#   - Layer-wise goodness separation (pos should be > neg)
#   - Validation accuracy improvement over epochs
#   - Loss decreasing over time
# ==============================================================================

history = train_ff(
    model=model,
    train_loader=train_loader,
    val_loader=val_loader,
    num_epochs=config.num_epochs,
    lr=config.learning_rate,
    hebbian_lr=config.hebbian_lr,
    device=DEVICE,
)

## Step 8: Visualize Training Progress

We plot the training metrics to understand how learning progressed.

In [ ]:
# ==============================================================================
# STEP 8: VISUALIZE TRAINING PROGRESS
# ==============================================================================
# This cell plots training metrics to visualize learning progress.
#
# Key plots:
#   1. Training loss over epochs
#   2. Validation accuracy over epochs
#   3. Layer-wise goodness (positive vs negative)
#   4. Layer-wise loss
# ==============================================================================

def plot_training_history(history: Dict, config: FFConfig):
    """Plot training metrics."""
    
    fig, axes = plt.subplots(2, 2, figsize=(14, 10))
    fig.suptitle('Forward-Forward Training Progress', fontsize=14, fontweight='bold')
    
    # 1. Training loss
    axes[0, 0].plot(history["train_loss"], 'b-', linewidth=2)
    axes[0, 0].set_xlabel('Epoch')
    axes[0, 0].set_ylabel('Loss')
    axes[0, 0].set_title('Training Loss')
    axes[0, 0].grid(True, alpha=0.3)
    
    # 2. Accuracy
    epochs_with_acc = [0] + list(range(4, len(history["train_loss"]), 5)) + [len(history["train_loss"])-1]
    epochs_with_acc = sorted(set(epochs_with_acc))
    if len(history["val_acc"]) > 0:
        axes[0, 1].plot(epochs_with_acc[:len(history["val_acc"])], history["val_acc"], 
                        'g-o', linewidth=2, markersize=6, label='Validation')
        axes[0, 1].plot(epochs_with_acc[:len(history["train_acc"])], history["train_acc"], 
                        'b-s', linewidth=2, markersize=6, label='Training')
        axes[0, 1].legend()
    axes[0, 1].set_xlabel('Epoch')
    axes[0, 1].set_ylabel('Accuracy')
    axes[0, 1].set_title('Classification Accuracy')
    axes[0, 1].set_ylim([0, 1.05])
    axes[0, 1].grid(True, alpha=0.3)
    
    # 3. Layer-wise goodness
    colors = plt.cm.viridis(np.linspace(0.2, 0.8, len(history["layer_pos_goodness"])))
    for i, (pos_g, neg_g) in enumerate(zip(history["layer_pos_goodness"], 
                                           history["layer_neg_goodness"])):
        axes[1, 0].plot(pos_g, color=colors[i], linestyle='-', 
                        linewidth=2, label=f'Layer {i} (pos)')
        axes[1, 0].plot(neg_g, color=colors[i], linestyle='--', 
                        linewidth=2, alpha=0.7)
    axes[1, 0].axhline(y=config.threshold, color='red', linestyle=':', 
                       linewidth=2, label=f'Threshold ({config.threshold})')
    axes[1, 0].set_xlabel('Epoch')
    axes[1, 0].set_ylabel('Goodness')
    axes[1, 0].set_title('Layer-wise Goodness (solid=pos, dashed=neg)')
    axes[1, 0].legend(loc='best', fontsize=8)
    axes[1, 0].grid(True, alpha=0.3)
    
    # 4. Layer-wise loss
    for i, layer_loss in enumerate(history["layer_losses"]):
        axes[1, 1].plot(layer_loss, color=colors[i], linewidth=2, label=f'Layer {i}')
    axes[1, 1].set_xlabel('Epoch')
    axes[1, 1].set_ylabel('Loss')
    axes[1, 1].set_title('Layer-wise FF Loss')
    axes[1, 1].legend(loc='best')
    axes[1, 1].grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()
    
    # Print summary
    print(f"\n{'='*60}")
    print("TRAINING SUMMARY")
    print(f"{'='*60}")
    if len(history["val_acc"]) > 0:
        print(f"Final validation accuracy: {history['val_acc'][-1]:.1%}")
        print(f"Best validation accuracy: {max(history['val_acc']):.1%}")
    print(f"\nFinal layer-wise goodness separation:")
    for i in range(len(history['layer_pos_goodness'])):
        pos = history['layer_pos_goodness'][i][-1]
        neg = history['layer_neg_goodness'][i][-1]
        sep = pos - neg
        print(f"  Layer {i}: pos={pos:.3f}, neg={neg:.3f}, separation={sep:+.3f}")


plot_training_history(history, config)

## Step 9: Visualize Predictions

Let's see how the model performs on individual samples.

In [ ]:
# ==============================================================================
# STEP 9: VISUALIZE PREDICTIONS
# ==============================================================================
# This cell shows model predictions on individual samples.
#
# For each sample, we show:
#   - The input signal
#   - The true label
#   - The predicted label
#   - Whether the prediction is correct (green) or wrong (red)
# ==============================================================================

def visualize_predictions(model: FFNetwork, dataset: Dataset, n_samples: int = 8):
    """Visualize model predictions on samples."""
    
    model.eval()
    
    # Select random samples
    indices = np.random.choice(len(dataset), min(n_samples, len(dataset)), replace=False)
    
    n_cols = min(4, n_samples)
    n_rows = (n_samples + n_cols - 1) // n_cols
    
    fig, axes = plt.subplots(n_rows, n_cols, figsize=(3.5*n_cols, 3*n_rows))
    if n_rows == 1:
        axes = axes.reshape(1, -1)
    
    fig.suptitle('Forward-Forward Predictions', fontsize=14, fontweight='bold')
    
    class_names = ["One Pulse", "Two Pulses"]
    correct_count = 0
    
    with torch.no_grad():
        for i, idx in enumerate(indices):
            sample = dataset[idx]
            signal = sample["signal"].unsqueeze(0).to(DEVICE)
            label = sample["label"].item()
            
            # Flatten and predict
            flat_signal = signal.view(1, -1)
            prediction = model.predict(flat_signal).item()
            
            is_correct = prediction == label
            correct_count += is_correct
            
            # Plot
            row, col = i // n_cols, i % n_cols
            ax = axes[row, col]
            
            ax.plot(sample["signal"][:, 0].numpy(), 'b-', linewidth=1.5)
            ax.set_ylim(-0.1, 1.1)
            ax.grid(True, alpha=0.3)
            
            color = 'green' if is_correct else 'red'
            symbol = '✓' if is_correct else '✗'
            
            ax.set_title(
                f"{symbol} Pred: {class_names[prediction]}\nTrue: {class_names[label]}",
                fontsize=9,
                color=color,
                fontweight='bold' if not is_correct else 'normal'
            )
            
            if col == 0:
                ax.set_ylabel("Signal")
            if row == n_rows - 1:
                ax.set_xlabel("Time step")
    
    # Hide empty subplots
    for i in range(n_samples, n_rows * n_cols):
        row, col = i // n_cols, i % n_cols
        axes[row, col].axis('off')
    
    plt.tight_layout()
    plt.show()
    
    print(f"\nSample accuracy: {correct_count}/{n_samples} ({correct_count/n_samples:.1%})")


visualize_predictions(model, val_dataset, n_samples=8)

## Step 10: Analyze Goodness Distributions

Let's see how well the layers separate positive and negative goodness.

In [ ]:
# ==============================================================================
# STEP 10: ANALYZE GOODNESS DISTRIBUTIONS
# ==============================================================================
# This cell analyzes how well each layer separates positive vs negative goodness.
#
# A well-trained layer should have:
#   - Positive goodness distribution shifted to the RIGHT
#   - Negative goodness distribution shifted to the LEFT
#   - Minimal overlap between the two distributions
# ==============================================================================

def analyze_goodness_distributions(model: FFNetwork, data_loader: DataLoader):
    """Analyze goodness distributions for positive vs negative data."""
    
    model.eval()
    
    all_pos_goodness = [[] for _ in range(len(model.layers))]
    all_neg_goodness = [[] for _ in range(len(model.layers))]
    
    with torch.no_grad():
        for batch in data_loader:
            pos_data = batch["positive"].to(DEVICE)
            neg_data = batch["negative"].to(DEVICE)
            
            pos_input = pos_data
            neg_input = neg_data
            
            for layer_idx, layer in enumerate(model.layers):
                # Forward and compute goodness
                pos_output = layer(pos_input)
                pos_g = layer.compute_goodness()
                
                neg_output = layer(neg_input)
                neg_g = layer.compute_goodness()
                
                all_pos_goodness[layer_idx].extend(pos_g.cpu().numpy())
                all_neg_goodness[layer_idx].extend(neg_g.cpu().numpy())
                
                pos_input = pos_output
                neg_input = neg_output
    
    # Plot distributions
    fig, axes = plt.subplots(1, len(model.layers), figsize=(6*len(model.layers), 5))
    if len(model.layers) == 1:
        axes = [axes]
    
    fig.suptitle('Goodness Distributions by Layer', fontsize=14, fontweight='bold')
    
    for i, ax in enumerate(axes):
        pos_g = np.array(all_pos_goodness[i])
        neg_g = np.array(all_neg_goodness[i])
        
        ax.hist(pos_g, bins=50, alpha=0.6, color='green', 
                label=f'Positive (μ={pos_g.mean():.2f})', density=True)
        ax.hist(neg_g, bins=50, alpha=0.6, color='red', 
                label=f'Negative (μ={neg_g.mean():.2f})', density=True)
        
        ax.axvline(x=config.threshold, color='black', linestyle='--', 
                   linewidth=2, label=f'Threshold ({config.threshold})')
        
        ax.set_xlabel('Goodness', fontsize=12)
        ax.set_ylabel('Density', fontsize=12)
        ax.set_title(f'Layer {i}', fontsize=12)
        ax.legend(loc='best')
        ax.grid(True, alpha=0.3)
        
        # Print separation stats
        separation = pos_g.mean() - neg_g.mean()
        print(f"Layer {i}: separation = {separation:.3f}")
    
    plt.tight_layout()
    plt.show()


analyze_goodness_distributions(model, val_loader)

## Summary

### What We Learned

1. **Forward-Forward eliminates backpropagation** by using local "goodness" objectives
2. **Each layer learns independently** - no gradient flow between layers
3. **Label embedding** concatenates the class label to the input
4. **Inference requires multiple passes** (one per class)

### Architecture Comparison: This Tutorial vs SOEN Backprop

| Aspect | SOEN (02_train_a_model) | FF (This Tutorial) |
|--------|--------------------------|---------------------|
| Hidden neurons | 5 (SingleDendrite) | **5** (FFLayer) |
| Input processing | Temporal (64 timesteps) | Flattened (66 dim) |
| Recurrence | Yes (J_1_to_1) | No |
| Learning | Backprop (global loss) | FF (local goodness) |
| Params | ~27 connection weights | ~335 weights |

**Note on parameter difference**: FF has more parameters because it processes the flattened
sequence (66 inputs) rather than 1 input per timestep. The hidden dimension (5) matches.

### Forward-Forward vs Backpropagation

| Aspect | Backpropagation | Forward-Forward |
|--------|-----------------|------------------|
| Gradient flow | Through all layers | None between layers |
| Learning signal | Global loss | Local goodness |
| Hardware fit | Poor | Good |
| Inference passes | 1 | N (one per class) |
| Accuracy | Higher | Lower (trade-off) |

### Hardware Compatibility

| Operation | Standard Mode | Hardware Mode |
|-----------|---------------|---------------|
| Goodness | mean(x²) | mean(\|x\|) |
| Normalization | L2 norm | Lateral inhibition |
| Loss | Softplus | Hinge |
| Update | Adam + autograd | Hebbian |

### Next Steps

- Try `HARDWARE_MODE = True` to see hardware-compatible training
- Experiment with different network sizes (hidden_dim)
- Compare results with the backprop version in `02_train_a_model.ipynb`
- Apply to MNIST (see Tutorial 04)